In [ ]:
import pandas as pd
import sys

# --- 1. Define filenames ---
emissions_filename = 'EDGAR v8.1.csv'
air_quality_filename = 'who.csv'


# --- 2. Load and preprocess emissions data (EDGAR) ---

try:
    # The EDGAR file has 9 descriptive header rows that need to be skipped
    emissions_df = pd.read_csv(emissions_filename, skiprows=9)
    print(f"Successfully loaded emissions data file: {emissions_filename}")

    # Filter for Black Carbon (BC) as our primary pollutant indicator
    emissions_df = emissions_df[emissions_df['Substance'] == 'BC']

    # Select the required columns: Country name, industry sector, and emissions data from 2000-2022
    cols_to_keep = ['Name', 'ipcc_code_2006_for_standard_report_name'] + [f'Y_{year}' for year in range(2000, 2023)]
    emissions_df = emissions_df[cols_to_keep]
    
    # Rename columns for easier handling
    emissions_df.rename(columns={'Name': 'Country', 'ipcc_code_2006_for_standard_report_name': 'Sector'}, inplace=True)

    # Convert the wide table to a long table (Melt)
    emissions_long = pd.melt(emissions_df, id_vars=['Country', 'Sector'], var_name='Year', value_name='Emissions')
    
    # Convert the 'Year' column from 'Y_2000' format to integer 2000
    emissions_long['Year'] = emissions_long['Year'].str.replace('Y_', '').astype(int)

    # Use pivot_table to convert each Sector into a feature column
    emissions_pivot = emissions_long.pivot_table(index=['Country', 'Year'], columns='Sector', values='Emissions').reset_index()
    
    print("Emissions data has been successfully reshaped.")

except FileNotFoundError:
    print(f"Error: File not found '{emissions_filename}'. Please ensure it is in the same directory as your Notebook.")
    sys.exit()
except Exception as e:
    print(f"An error occurred while processing emissions data: {e}")
    sys.exit()


# --- 3. Load and preprocess air quality data (WHO) ---
try:
    air_quality_df = pd.read_csv(air_quality_filename, encoding='latin1') # Use latin1 encoding to avoid errors
    print(f"Successfully loaded air quality data file: {air_quality_filename}")

    # Select and rename the required columns
    air_quality_df = air_quality_df[['country_name', 'year', 'pm25_concentration']]
    air_quality_df.rename(columns={'country_name': 'Country', 'year': 'Year', 'pm25_concentration': 'PM2.5'}, inplace=True)

    # Clean the data: remove rows with missing PM2.5 and convert the PM2.5 column to a numeric type
    air_quality_df.dropna(subset=['PM2.5'], inplace=True)
    air_quality_df['PM2.5'] = pd.to_numeric(air_quality_df['PM2.5'], errors='coerce')
    air_quality_df.dropna(subset=['PM2.5'], inplace=True)

    # Aggregate data: calculate the mean PM2.5 for each country per year (as the original data might be at the city level)
    country_annual_pm25 = air_quality_df.groupby(['Country', 'Year'])['PM2.5'].mean().reset_index()
    
    print("Air quality data has been successfully aggregated to the country-year level.")

except FileNotFoundError:
    print(f"Error: File not found '{air_quality_filename}'. Please ensure it is in the same directory as your Notebook.")
    sys.exit()
except Exception as e:
    print(f"An error occurred while processing air quality data: {e}")
    sys.exit()


# --- 4. Merge the two datasets ---
print("\nMerging emissions and air quality data...")
# Use an inner join to ensure only countries and years present in both datasets are kept
final_df = pd.merge(emissions_pivot, country_annual_pm25, on=['Country', 'Year'], how='inner')

# Fill all remaining missing values (mainly from the pivot operation) with 0
final_df.fillna(0, inplace=True)

print(f"Data successfully merged. The final dataset contains {final_df.shape[0]} rows and {final_df.shape[1]} columns.")


# --- 5. Save the final file ---
output_filename = 'final_data_for_modeling.csv'
final_df.to_csv(output_filename, index=False)

print(f"\nProcessing complete! The final data for modeling has been saved as '{output_filename}'")